<a href="https://colab.research.google.com/github/ayubginting1997/Compiler-Front-End-Python-Implementation/blob/master/Tubes_Hypoxia_Monitoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PPG memanfaatkan Flash dan Camera Handphone


In [ ]:
import cv2
import numpy as np
from scipy import signal
import os
import shutil
"""
# This Function Works as follows:
1. Takes input as the path of the finger video 
2. Returns the resting heartrate in bpm as a number 


- It first gets frames from video at rate 20 frames per second
- Extract channels from the images 
- then it computes the average of the red channels and saves them in pixels_averages
- Then do a HPF to the average
- Truncate wonky data from  beginning of series
- Take FFT to the filtered data and the compute the heartrate in bpm.
"""
overall_results = []

def HeartRateFinger(Video_Path):
    os.makedirs("frames")  #make a directory called frames to save the images in
    vidcap = cv2.VideoCapture(Video_Path)  #the directory path to the video of interest
    images = []

    #take in images and get indidvual frames
    def getFrame(sec):
        vidcap.set(cv2.CAP_PROP_POS_MSEC, sec * 1000)  #VideoCaptureProperties to capture 
        hasFrames, image = vidcap.read()
        if hasFrames:
            cv2.imwrite("frames/image" + str(count) + ".jpg", image)  # save frame as JPG file
        images.append("frames/image" + str(count) + ".jpg")
        return hasFrames

    #set up frame rate
    sec = 0
    frameRate = 0.05  #capture frame each 0.05 seconds --> (1 second) / .05 = 20 seconds 20 fps
    count = 1
    success = getFrame(sec)  #success tracker
    while (success):
        count = count + 1
        sec = sec + frameRate
        sec = round(sec, 2)
        success = getFrame(sec)

    #Get the channels from images and Find the average of RED channel for each frame
    pixels_averages = []
    for i in range(len(images) - 1):
        bgr_image = cv2.imread(images[i])
        blue_channel, green_channel, red_channel = cv2.split(bgr_image)  #splits into the 3 color channels (RGB)
        average = np.mean(red_channel)  #mean of all the red channels  is the sum all pixels in channel and divide by number of pixels
        pixels_averages.append(average)  #add this average to all the other frames array
    pixels_averages = np.divide(np.array(pixels_averages), 255)  #normalize our averages between 0-1

    #Clean and filter data (butterworth filter)
    sos = signal.butter(4, 1, 'hp', fs=20, output='sos')  # setting up signal.butter(kind of frequency,framerate, name of method)
    filtered = signal.sosfilt(sos, pixels_averages) #Applying filter

    # truncate wonky data from  beginning of series
    filtered = filtered[40:]

    #Perform FFT to find frequency of max amplitude
    Sample_rate = 20  #same as the frame rate is the sampling rate
    BW = Sample_rate / 2  #bandwidth (range of frequency) in signal processing
    fft = np.absolute(np.fft.fft(filtered))
    frames_len = len(filtered)
    frequancies = np.arange(0, BW, Sample_rate / frames_len)
    fft = fft[0:len(frequancies)]

    #Convert back to BPM from hz
    heartrate = np.round(frequancies[np.argmax(fft)] * 60)  #take the highest peak frequency and multiply by 60 and round it
    shutil.rmtree("frames")  #remove folder made to keep the directory clean
    overall_results.append(heartrate)
    return heartrate



In [ ]:
#script expected outcome while Resting is 74-78 BPM
path = "/content/sample_data/95.MOV"
my_heart_rate = HeartRateFinger(path)
print("Your Heart rate is : {} bpm".format(my_heart_rate))

Your Heart rate is : 77.0 bpm


In [ ]:
#script expected outcome while Resting is 69-77 BPM
path = "/content/sample_data/resting1.mp4"
my_heart_rate = HeartRateFinger(path)
print("Your Heart rate is : {} bpm".format(my_heart_rate))

Your Heart rate is : 70.0 bpm


In [ ]:
#script expected outcome while Resting is 69-77 BPM
path = "/content/sample_data/RESTING2.mp4"
my_heart_rate = HeartRateFinger(path)
print("Your Heart rate is : {} bpm".format(my_heart_rate))

Your Heart rate is : 70.0 bpm


In [ ]:
#script expected outcome after activity was 83-88 BPM (50 jumping jacks)
path = "/content/sample_data/active1.mp4"
my_heart_rate = HeartRateFinger(path)
print("Your Heart rate is : {} bpm".format(my_heart_rate))

Your Heart rate is : 85.0 bpm


In [ ]:
#script expected outcome after activity was 75-85 BPM (jogging for a bit)
path =  "/content/sample_data/active2.mp4"
my_heart_rate = HeartRateFinger(path)
print("Your Heart rate is : {} bpm".format(my_heart_rate))

Your Heart rate is : 75.0 bpm


In [ ]:
#Overall results laid out with their error rate as well
print(overall_results)


[77.0, 85.0]


# PPG PREDICTION 

In [ ]:
import pandas as pd
from calcul_features import calcul_features
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut
from sklearn.tree import DecisionTreeRegressor

# chargement des labels
SBP_label = pd.read_csv("/media/PPGonMObile/DATASET/SBP_label.csv", delimiter = ';', header = None)
DBP_label = pd.read_csv("/media/PPGonMObile/DATASET/DBP_label.csv", delimiter = ';', header = None)

# chargement des fichiers segmentés
fichier_segmente = pd.read_csv("/media/PPGonMObile/DATASET/segmente.csv", delimiter = ';', header = None)
# chargement des features liés à l'anthropometrie du sujet : age, poids, taille, sex
features_anthropo = pd.read_csv("/media/PPGonMObile/DATASET/features_anthropo.csv", delimiter = ';')

BUFFER_SIZE = 1200

# on calcule tous les features liés au signal et on ajoute aussi les features anthropométriques au même tableau de features
features = calcul_features(fichier_segmente,features_anthropo,SBP_label,BUFFER_SIZE)

# permet de choisir quels features on souhaite garder pour l'entrainement du modèles
features = pd.concat([features.PPG_mean, features.PPG_min,features.PPG_max,features.PPG_amp,features.PPG_q_0_75, 
                                    features.PPG_q_0_25,features.PPG_0_cross,features.PPG_kurt,features.PPG_var,features.PPG_len, 
                                    features.PPG_S_len, features.PPG_D_len,features.PPG_int, features.PPG_S_int, features.PPG_D_int,
                                    features.PPG_len_S_tot,features.PPG_len_D_tot, features.PPG_len_D_S, 
                                    features.PPG_int_S_tot, features.PPG_int_D_tot, features.PPG_int_D_S,features.PPG_max_D, 
                                    features.PPG_S_High_0_10, features.PPG_S_High_0_25,features.PPG_S_High_0_33,features.PPG_S_High_0_50,
                                    features.PPG_S_High_0_66,features.PPG_S_High_0_75,
                                    features.PPG_D_High_0_10, features.PPG_D_High_0_25,features.PPG_D_High_0_33,features.PPG_D_High_0_50,
                                    features.PPG_D_High_0_66,features.PPG_D_High_0_75,
                                    features.sex, features.age, features.weight, features.height, features.bmi], axis =1)

X_train = features.values

# le signal est dejà normalisé mais il est important de normaliser/scaler les features entre eux
max_abs_scaler = preprocessing.MaxAbsScaler()
#min_max_scaler = preprocessing.MinMaxScaler()
X_train = max_abs_scaler.fit_transform(X_train)
#X_train = min_max_scaler.fit_transform(X_train)
#X_train = normalize(X_train, norm = 'max')
"""sc = StandardScaler()
X_train = sc.fit_transform(X_train)"""

# on divise les données en données d'entrainement et de test 
X_train, X_test, y_train, y_test = train_test_split(X_train, SBP_label, test_size=0.2, random_state=0)

# utiliser de gridsearchCV pour extraire les paramètres les plus appropriés
# intéressant aussi car cross validation auto

#random forest regressor
params_rfr = {
    "n_estimators": [5,10,15,20,25,30,35],
    "max_depth" : [10,15,20,25,30],
    "random_state" : [0]
}
rfr = RandomForestRegressor()
gsc_rfr = GridSearchCV(rfr, params_rfr, cv=7)

gsc_rfr.fit(X_train, np.ravel(y_train))
y_pred = gsc_rfr.best_estimator_.predict(X_test)

# decision tree regressor
"""dtr = DecisionTreeRegressor()
params_dtr = {
    "max_depth" : [10,15,20,25,30],
}
gsc_dtr = GridSearchCV(dtr, params_dtr, cv=7)
gsc_dtr.fit(X_train, np.ravel(y_train))
y_pred = gsc_dtr.best_estimator_.predict(X_test)"""

# support vector regressor
"""params_svr = {
    "kernel": ['linear'],
    "C" : [100],
    "gamma" : ['auto']
}
svr = SVR()
gsc_svr = GridSearchCV(svr, params_svr, cv=7)
gsc_svr.fit(X_train, np.ravel(y_train))
y_pred = gsc_svr.best_estimator_.predict(X_test)"""

# adaboost regressor
'''adb = AdaBoostRegressor(random_state=0, n_estimators=100)
adb.fit(X_train, y_train)
y_pred = adb.predict(X_test)'''

"""adb = AdaBoostRegressor(random_state=0, n_estimators=100)
adb.fit(X_train, y_train)
y_pred = adb.predict(X_test)"""

# linear regression
"""lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)"""

# decision tree regression
"""dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train)
y_pred = dtr.predict(X_test)"""

y_pred = y_pred.reshape((-1,1))
y_test = y_test.values

error_moy = np.mean(y_pred-y_test)
std = np.std(y_pred-y_test)

plt.figure()
plt.plot(y_test, y_pred, 'bo')
plt.title('SBP predite en fonction de SBP réelle')
plt.xlabel('SBP réelle')
plt.ylabel('SBP prédite')

plt.show()

print('Erreur moyenne',error_moy)
print('Ecart type',std)

ModuleNotFoundError: ignored

# CALCULATION PPG

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy import integrate
import matplotlib.pyplot as plt

def calcul_features(PPG,features_anthropo, Ytrain,BUFFER_SIZE):
    #calcul des features pour la prediction
    PPG = PPG.values
    feature = pd.DataFrame(index = Ytrain, columns = ['PPG_mean', 'PPG_min','PPG_max','PPG_std','PPG_amp','PPG_q_0_75', 
                                        'PPG_q_0_25','PPG_0_cross','PPG_kurt','PPG_skew','PPG_var','PPG_len', 'PPG_S_len', 'PPG_D_len',
                                        'PPG_int', 'PPG_S_int', 'PPG_D_int','PPG_len_S_tot','PPG_len_D_tot', 'PPG_len_D_S', 
                                        'PPG_int_S_tot', 'PPG_int_D_tot', 'PPG_int_D_S','PPG_max_D', 
                                        'PPG_S_High_0_10', 'PPG_S_High_0_25','PPG_S_High_0_33','PPG_S_High_0_50','PPG_S_High_0_66','PPG_S_High_0_75',
                                        'PPG_D_High_0_10', 'PPG_D_High_0_25','PPG_D_High_0_33','PPG_D_High_0_50','PPG_D_High_0_66','PPG_D_High_0_75',
                                        'sex', 'age', 'weight', 'height', 'bmi',
                                        'D_PPG_mean','D_PPG_min','D_PPG_max','D_PPG_std','D_PPG_amp','D_PPG_q_0_75', 
                                        'D_PPG_q_0_25','D_PPG_0_cross','D_PPG_kurt','D_PPG_skew'])
    
    PPG_mean = np.zeros(PPG.shape[0]) # moyenne signal
    PPG_min = np.zeros(PPG.shape[0]) # minimum signal
    PPG_max = np.zeros(PPG.shape[0]) # max signal
    PPG_std = np.zeros(PPG.shape[0]) # ecart type signal
    PPG_amp = np.zeros(PPG.shape[0]) # amplitude signal
    PPG_0_75 = np.zeros(PPG.shape[0]) # percentile 0.75 signal
    PPG_0_25 = np.zeros(PPG.shape[0]) # percentile 0.25 signal
    PPG_0_cross = np.zeros(PPG.shape[0]) # nombre d'intersection à 0
    PPG_kurt = np.zeros(PPG.shape[0]) # kurtosis signal
    PPG_skew = np.zeros(PPG.shape[0]) # skewness signal
    PPG_var = np.zeros(PPG.shape[0]) # variance signal
    PPG_len = np.zeros(PPG.shape[0]) # longueur du signal
    PPG_S_len = np.zeros(PPG.shape[0]) # longueur temps systolique
    PPG_D_len = np.zeros(PPG.shape[0]) # lo,gueur temps diastolique
    PPG_int = np.zeros(PPG.shape[0]) # integrale signal
    PPG_S_int = np.zeros(PPG.shape[0]) # integrale aire systolique
    PPG_D_int = np.zeros(PPG.shape[0]) # integrale aire diastolique
    PPG_len_S_tot = np.zeros(PPG.shape[0]) # rapppport temps systolique/temps total
    PPG_len_D_tot = np.zeros(PPG.shape[0]) # rapppport temps diastolique/temps total
    PPG_len_D_S = np.zeros(PPG.shape[0]) # rapppport temps diastolique/temps systolique
    PPG_int_S_tot = np.zeros(PPG.shape[0]) # rapppport integrale systolique/integrale total
    PPG_int_D_tot = np.zeros(PPG.shape[0]) # rapppport integrale diastolique/integrale total
    PPG_int_D_S = np.zeros(PPG.shape[0]) # rapppport integrale diastolique/integrale systolique
    PPG_max_D = np.zeros(PPG.shape[0]) # max de la dérivéé du signal
    PPG_S_High_0_10 = np.zeros(PPG.shape[0]) # abscisse à 0.10*max_abs côté gauche
    PPG_S_High_0_25 = np.zeros(PPG.shape[0]) # abscisse à 0.25*max_abs côté gauche
    PPG_S_High_0_33 = np.zeros(PPG.shape[0]) # abscisse à 0.33*max_abs côté gauche
    PPG_S_High_0_50 = np.zeros(PPG.shape[0]) # abscisse à 0.50*max_abs côté gauche
    PPG_S_High_0_66 = np.zeros(PPG.shape[0]) # abscisse à 0.66*max_abs côté gauche
    PPG_S_High_0_75 = np.zeros(PPG.shape[0]) # abscisse à 0.75*max_abs côté gauche
    
    PPG_D_High_0_10 = np.zeros(PPG.shape[0]) # abscisse à 0.10*max_abs côté droit
    PPG_D_High_0_25 = np.zeros(PPG.shape[0]) # abscisse à 0.25*max_abs côté droit
    PPG_D_High_0_33 = np.zeros(PPG.shape[0]) # abscisse à 0.33*max_abs côté droit
    PPG_D_High_0_50 = np.zeros(PPG.shape[0]) # abscisse à 0.50*max_abs côté droit
    PPG_D_High_0_66 = np.zeros(PPG.shape[0]) # abscisse à 0.66*max_abs côté droit
    PPG_D_High_0_75 = np.zeros(PPG.shape[0]) # abscisse à 0.75*max_abs côté droit
    
    D_PPG = np.zeros((PPG.shape[0],PPG.shape[1])) # creation du vecteur derivee du signal
    D_PPG_mean = np.zeros(PPG.shape[0]) # moyenne derivee signal
    D_PPG_min = np.zeros(PPG.shape[0]) # min derivee signal
    D_PPG_max = np.zeros(PPG.shape[0]) # max derivee signal 
    D_PPG_std = np.zeros(PPG.shape[0]) # ecart type derivee signal
    D_PPG_amp = np.zeros(PPG.shape[0]) # amplitude derivee
    D_PPG_0_75 = np.zeros(PPG.shape[0]) # percentile 0.75 derivee
    D_PPG_0_25 = np.zeros(PPG.shape[0]) # percentile 0.25 derivee
    D_PPG_0_cross = np.zeros(PPG.shape[0]) # intersections à 0 de la derivee
    D_PPG_kurt = np.zeros(PPG.shape[0]) # kurtosis de la derivee
    D_PPG_skew = np.zeros(PPG.shape[0]) # skewness de la derivee
    
    for i in range(PPG.shape[0]):
        cpt = 1
        # Buffer_size doit être égal au size_exp du fichier read.py
        # la fin du signal est composée de 0, on revient en arrière jusqu'à ce qu'il n'y est plus de 0 pour
        # connaitre la taille exacte du signal
        # une fois celle-ci connue on peut calculer les features sur le signal
        while PPG[i,BUFFER_SIZE - cpt] == PPG[i,BUFFER_SIZE-1]:
            cpt = cpt + 1
            
        """if i < 10:
            plt.figure()
            plt.plot(PPG[i,0:BUFFER_SIZE - cpt])
            plt.show()"""
            
        PPG_mean[i] =  np.mean(PPG[i,0:BUFFER_SIZE - cpt])
        PPG_min[i] =  np.min(PPG[i,0:BUFFER_SIZE - cpt])
        PPG_max[i] =  np.max(PPG[i,0:BUFFER_SIZE - cpt])
        PPG_std[i] =  np.std(PPG[i,0:BUFFER_SIZE - cpt])
        PPG_amp[i] =  np.max(PPG[i,0:BUFFER_SIZE - cpt]) - np.min(PPG[i,0:BUFFER_SIZE - cpt])
        PPG_0_75[i] = np.quantile(PPG[i,0:BUFFER_SIZE - cpt], 0.75)
        PPG_0_25[i] = np.quantile(PPG[i,0:BUFFER_SIZE - cpt], 0.25)
        PPG_0_cross[i] = len(np.where(np.diff(np.sign(PPG[i,0:BUFFER_SIZE - cpt])))[0])
        PPG_kurt[i] = kurtosis(PPG[i,0:BUFFER_SIZE - cpt])
        PPG_skew[i] =  skew(PPG[i,0:BUFFER_SIZE - cpt])
        
        if np.mean(PPG[i,0:BUFFER_SIZE - cpt]) == 0:
            PPG_var[i] =  float(np.std(PPG[i,0:BUFFER_SIZE - cpt]))
        else:
            PPG_var[i] =  float(np.std(PPG[i,0:BUFFER_SIZE - cpt])) / float(np.mean(PPG[i,0:BUFFER_SIZE - cpt]))
        
        PPG_len[i] = BUFFER_SIZE-cpt
        PPG_S_len[i] = np.argmax(PPG[i,0:BUFFER_SIZE - cpt])
        PPG_D_len[i] = BUFFER_SIZE - cpt - np.argmax(PPG[i,0:BUFFER_SIZE - cpt])
        PPG_int[i] = np.trapz(PPG[i,0:BUFFER_SIZE - cpt], dx = 1/1000)
        PPG_S_int[i] = np.trapz(PPG[i,0:np.argmax(PPG[i,0:BUFFER_SIZE - cpt])], dx = 1/1000)
        PPG_D_int[i] = np.trapz(PPG[i,np.argmax(PPG[i,0:BUFFER_SIZE - cpt]):BUFFER_SIZE - cpt], dx = 1/1000)
        PPG_len_S_tot[i] = PPG_S_len[i]/PPG_len[i]
        PPG_len_D_tot[i] = PPG_D_len[i]/PPG_len[i]
        PPG_len_D_S[i] = PPG_D_len[i]/PPG_S_len[i]
        PPG_int_S_tot[i] = PPG_S_int[i]/PPG_int[i]
        PPG_int_D_tot[i] = PPG_D_int[i]/PPG_int[i]
        PPG_int_D_S[i] = PPG_D_int[i]/PPG_S_int[i]
        PPG_max_D[i] = np.max(np.gradient(PPG[i,0:BUFFER_SIZE - cpt], 1/1000))
        
        for k in range(np.argmax(PPG[i,0:BUFFER_SIZE - cpt])):
            if PPG[i,k] < 0.10 * PPG_max[i] and PPG[i,k+1] > 0.10 * PPG_max[i]:
                PPG_S_High_0_10[i] = np.argmax(PPG[i,0:BUFFER_SIZE - cpt]) - k
                
            if PPG[i,k] < 0.25 * PPG_max[i] and PPG[i,k+1] > 0.25 * PPG_max[i]:
                PPG_S_High_0_25[i] = np.argmax(PPG[i,0:BUFFER_SIZE - cpt]) - k
            
            if PPG[i,k] < 0.33 * PPG_max[i] and PPG[i,k+1] > 0.33 * PPG_max[i]:
                PPG_S_High_0_33[i] = np.argmax(PPG[i,0:BUFFER_SIZE - cpt]) - k
            
            if PPG[i,k] < 0.50 *PPG_max[i] and PPG[i,k+1] > 0.50 * PPG_max[i]:
                PPG_S_High_0_50[i] = np.argmax(PPG[i,0:BUFFER_SIZE - cpt]) - k
                
            if PPG[i,k] < 0.66 *PPG_max[i] and PPG[i,k+1] > 0.66 * PPG_max[i]:
                PPG_S_High_0_66[i] = np.argmax(PPG[i,0:BUFFER_SIZE - cpt]) - k
            
            if PPG[i,k] < 0.75 * PPG_max[i] and PPG[i,k+1] > 0.75 * PPG_max[i]:
                PPG_S_High_0_75[i] = np.argmax(PPG[i,0:BUFFER_SIZE - cpt]) - k
        
        
        for k in range(np.argmax(PPG[i,0:BUFFER_SIZE - cpt]),BUFFER_SIZE - cpt):
            if PPG[i,k] > 0.10 * PPG_max[i]and PPG[i,k+1] < 0.10 * PPG_max[i]:
                PPG_D_High_0_10[i] = k - np.argmax(PPG[i,0:BUFFER_SIZE - cpt])
                
            if PPG[i,k] > 0.25 * PPG_max[i] and PPG[i,k+1] < 0.25 * PPG_max[i]:
                PPG_D_High_0_25[i] = k - np.argmax(PPG[i,0:BUFFER_SIZE - cpt])
            
            if PPG[i,k] > 0.33 * PPG_max[i] and PPG[i,k+1] < 0.33 * PPG_max[i]:
                PPG_D_High_0_33[i] = k - np.argmax(PPG[i,0:BUFFER_SIZE - cpt])
            
            if PPG[i,k] > 0.50 * PPG_max[i] and PPG[i,k+1] < 0.50 * PPG_max[i]:
                PPG_D_High_0_50[i] = k - np.argmax(PPG[i,0:BUFFER_SIZE - cpt])
                
            if PPG[i,k] > 0.66 * PPG_max[i] and PPG[i,k+1] < 0.66 * PPG_max[i]:
                PPG_D_High_0_66[i] = k - np.argmax(PPG[i,0:BUFFER_SIZE - cpt])
            
            if PPG[i,k] > 0.75 * PPG_max[i] and PPG[i,k+1] < 0.75 * PPG_max[i]:
                PPG_D_High_0_75[i] = k - np.argmax(PPG[i,0:BUFFER_SIZE - cpt])
                
        D_PPG[i,:] = np.gradient(PPG[i,:], 1/1000)
        """print(D_PPG)
        plt.figure()
        plt.plot(D_PPG)
        plt.show()"""
        
        D_PPG_mean[i] =  np.mean(D_PPG[i,0:BUFFER_SIZE - cpt])
        D_PPG_min[i] =  np.min(D_PPG[i,0:BUFFER_SIZE - cpt])
        D_PPG_max[i] =  np.max(D_PPG[i,0:BUFFER_SIZE - cpt])
        D_PPG_std[i] =  np.std(D_PPG[i,0:BUFFER_SIZE - cpt])
        D_PPG_amp[i] =  np.max(D_PPG[i,0:BUFFER_SIZE - cpt]) - np.min(PPG[i,0:BUFFER_SIZE - cpt])
        D_PPG_0_75[i] = np.quantile(D_PPG[i,0:BUFFER_SIZE - cpt], 0.75)
        D_PPG_0_25[i] = np.quantile(D_PPG[i,0:BUFFER_SIZE - cpt], 0.25)
        D_PPG_0_cross[i] = len(np.where(np.diff(np.sign(D_PPG[i,0:BUFFER_SIZE - cpt])))[0])
        D_PPG_kurt[i] = kurtosis(D_PPG[i,0:BUFFER_SIZE - cpt])
        D_PPG_skew[i] =  skew(D_PPG[i,0:BUFFER_SIZE - cpt])
        
    feature.PPG_mean = PPG_mean
    feature.PPG_min = PPG_min
    feature.PPG_max = PPG_max
    feature.PPG_std = PPG_std
    feature.PPG_amp = PPG_amp
    feature.PPG_q_0_75 = PPG_0_75
    feature.PPG_q_0_25 = PPG_0_25
    feature.PPG_0_cross = PPG_0_cross
    feature.PPG_kurt = PPG_kurt
    feature.PPG_skew = PPG_skew
    feature.PPG_var = PPG_var
    feature.PPG_len = PPG_len
    feature.PPG_S_len = PPG_S_len
    feature.PPG_D_len = PPG_D_len
    feature.PPG_int = PPG_int
    feature.PPG_S_int = PPG_S_int
    feature.PPG_D_int = PPG_D_int
    feature.PPG_len_S_tot = PPG_len_S_tot
    feature.PPG_len_D_tot = PPG_len_D_tot
    feature.PPG_len_D_S = PPG_len_D_S
    feature.PPG_int_S_tot = PPG_int_S_tot
    feature.PPG_int_D_tot = PPG_int_D_tot
    feature.PPG_int_D_S = PPG_int_D_S
    feature.PPG_max_D = PPG_max_D
    
    feature.PPG_S_High_0_10 = PPG_S_High_0_10
    feature.PPG_S_High_0_25 = PPG_S_High_0_25
    feature.PPG_S_High_0_33 = PPG_S_High_0_33
    feature.PPG_S_High_0_50 = PPG_S_High_0_50
    feature.PPG_S_High_0_66 = PPG_S_High_0_66
    feature.PPG_S_High_0_75 = PPG_S_High_0_75
    
    feature.PPG_D_High_0_10 = PPG_D_High_0_10
    feature.PPG_D_High_0_25 = PPG_D_High_0_25
    feature.PPG_D_High_0_33 = PPG_D_High_0_33
    feature.PPG_D_High_0_50 = PPG_D_High_0_50
    feature.PPG_D_High_0_66 = PPG_D_High_0_66
    feature.PPG_D_High_0_75 = PPG_D_High_0_75
    
    feature.D_PPG_mean = D_PPG_mean
    feature.D_PPG_min = D_PPG_min
    feature.D_PPG_max = D_PPG_max
    feature.D_PPG_std = D_PPG_std
    feature.D_PPG_amp = D_PPG_amp
    feature.D_PPG_q_0_75 = D_PPG_0_75
    feature.D_PPG_q_0_25 = D_PPG_0_25
    feature.D_PPG_0_cross = D_PPG_0_cross
    feature.D_PPG_kurt = D_PPG_kurt
    feature.D_PPG_skew = D_PPG_skew
    
    feature.sex = features_anthropo['feat_sex'].values
    feature.age = features_anthropo['feat_age'].values
    feature.weight = features_anthropo['feat_weight'].values
    feature.height = features_anthropo['feat_height'].values
    feature.bmi = features_anthropo['feat_bmi'].values
    
    ## fin calcul features
    
    return feature

# WAVELET TRANSFORM

DWT initial

In [ ]:
import numpy as np
import statistics
import pywt
from scipy.stats import kurtosis

def comp_moment(feature):
    step = int(len(feature)/2)
    avg_temp = np.zeros([2])
    stn_dev_temp = np.zeros([2])
    kurto_temp = np.zeros([2])
    for i in range(int(len(feature)/step)):
        avg_temp[i] = np.mean(feature[step*i:step*(i+1)])
        stn_dev_temp[i] = statistics.stdev(feature[step*i:step*(i+1)])
        kurto_temp[i] = kurtosis(feature[step*i:step*(i+1)])
        return (avg_temp, stn_dev_temp, kurto_temp)

def dwt(y):
    [a, d1, d2, d3, d4] = pywt.wavedec(y, 'haar', level=4)
    
    #Approximation coeficient
    avg_temp, stn_dev_temp, kurto_temp = comp_moment(a)
    avg = avg_temp
    stn_dev = stn_dev_temp
    kurto = kurto_temp

    # d1 coffiecient
    avg_temp, stn_dev_temp, kurto_temp = comp_moment(d1)
    avg = np.append(avg, avg_temp)
    stn_dev = np.append(stn_dev, stn_dev_temp)
    kurto = np.append(kurto, kurto_temp)

    # d2 coffiecient
    avg_temp, stn_dev_temp, kurto_temp = comp_moment(d2)
    avg = np.append(avg, avg_temp)
    stn_dev = np.append(stn_dev, stn_dev_temp)
    kurto = np.append(kurto, kurto_temp)

    # d3 coffiecient
    avg_temp, stn_dev_temp, kurto_temp = comp_moment(d3)
    avg = np.append(avg, avg_temp)
    stn_dev = np.append(stn_dev, stn_dev_temp)
    kurto = np.append(kurto, kurto_temp)

    # d4 coffiecient
    avg_temp, stn_dev_temp, kurto_temp = comp_moment(d4)
    avg = np.append(avg, avg_temp)
    stn_dev = np.append(stn_dev, stn_dev_temp)
    kurto = np.append(kurto, kurto_temp)

    feature_dwt = np.append(np.append(avg, stn_dev), kurto)

    return feature_dwt

Libraries Initial

In [ ]:
import pywt
import numpy as np 
import pandas as pd
from scipy.signal import find_peaks, resample
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


#variable global
sample_rate = 500
scaler = MinMaxScaler()

"""
Fungsi Lain
"""
def plot_color_text(filtered,predicted_beats,start_stop):
    minimum = min(filtered)
    maximum = max(filtered)
    lel = [np.arange(data[0],data[1]) for data in start_stop]
    #plt.plot(ecg_raw)
    plt.plot(filtered)
    for i in range(len(predicted_beats)):
        if (predicted_beats[i] != "N"):
            plt.fill_between(lel[i],minimum,maximum,facecolor='red', alpha=0.5)
            plt.text(start_stop[i][0],maximum,predicted_beats[i])
#    plt.scatter(peaks, [filtered[peaks[i]] for i in range(len(peaks))],c='red')
    plt.show()
    
def plot_with_rpeaks(filtered,r_peaks):
    peaks = [filtered[peak] for peak in r_peaks]
    plt.plot(filtered)
    plt.scatter(r_peaks,peaks,c='red')
#    plt.savefig("sinyal_clean_peaks",dpi=1000)
    plt.show()
"""
END FUNGSI LAIN
"""

"""
PREPROCESSING
"""
def find_signal_peaks(arraynya, minimum=0, maximum=None, freq=500):
    dist = freq/2
    r_peaks = find_peaks(arraynya,distance=dist,prominence=(minimum,maximum))
    return r_peaks[0].tolist()

def get_rr(r_peaks, to_sec=False,sample_rate=125):
    rr_list = []
    start_stop = []
    for i in range(len(r_peaks)-2):
        rr_list.append(r_peaks[i+1]-r_peaks[i])
        start_stop.append([r_peaks[i],r_peaks[i+1]])
    if (to_sec):
        rr_list = np.divide(rr_list,sample_rate)
    return rr_list, start_stop

def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def baseline_remove_ppg(signal,frequency=500):
    half_freq = int(frequency/2)
    baseline = resample(moving_average(signal,n=half_freq),len(signal))
    return np.subtract(signal,baseline)

def remove_signal(signal):
    return np.zeros_like(signal)

def remove_noise(noisy_signal):
    denoised = noisy_signal
    for i in range(5):
        denoised = pywt.dwt(denoised,'sym8')[0]
    for i in range(5):
        denoised = pywt.idwt(denoised,remove_signal(denoised),wavelet='sym8')
    return denoised

def annotation_to_ppg_signal_split(signal_path, annotation_path, number):
    signal_file = "%s/%s.csv" % (signal_path, number)
    anot_file = "%s/%s.csv" % (annotation_path, number)
    signal = pd.read_csv(signal_file, low_memory=False).replace("-",100)
    annotation = pd.read_csv(anot_file, low_memory=False)
    ppg = signal.iloc[1:,2].values
    ppg_normal, ppg_pac, ppg_pvc = [], [], []
    start, stop, signal_class = annotation["start_index"].values, annotation["stop_index"].values, annotation["signal_class"].values.tolist()
    for i in range(len(annotation)):
        if (signal_class[i] == "N"):
            ppg_normal.append(ppg[start[i]:stop[i]].astype("float64"))
        elif (signal_class[i] == "A"):
            ppg_pac.append(ppg[start[i]:stop[i]].astype("float64"))
        elif (signal_class[i] == "V"):
            ppg_pvc.append(ppg[start[i]:stop[i]].astype("float64"))
    return ppg_normal, ppg_pac, ppg_pvc

def annotation_to_ppg_signal_labeled(signal_path, annotation_path, number):
    signal_file = "%s/%s.csv" % (signal_path, number)
    anot_file = "%s/%s.csv" % (annotation_path, number)
    signal = pd.read_csv(signal_file, low_memory=False).replace("-",100)
    annotation = pd.read_csv(anot_file, low_memory=False)
    ppg = signal.iloc[1:,2].values
    ppg_signal = []
    start, stop, signal_class = annotation["start_index"].values, annotation["stop_index"].values, annotation["signal_class"].values.tolist()
    for i in range(len(annotation)):
            ppg_cut = ppg[start[i]:stop[i]].astype("float64").tolist()
            ppg_signal.append(ppg_cut)
    return ppg_signal, signal_class


def read_feature_from_csv(file_name):
    feature = pd.read_csv(file_name,low_memory=False).values
    pp_list = feature[:,0:-1]
    class_list = feature[:,-1]
    return pp_list, class_list

def read_feature_from_csv1(file_name):
    feature = pd.read_csv(file_name,low_memory=False)
    pp_list = feature.iloc[:,0:-1]
    class_list = feature.iloc[:,-1]
    return pp_list, class_list
    
def preprocess_ppg_signal(input_signal,sample_rate=500):
    baseline_removed = baseline_remove_ppg(input_signal,frequency=sample_rate)
    clean_ppg = remove_noise(baseline_removed)
    return clean_ppg

    
    
    

"""
END PREPROCESSING
"""    

"""
FITUR EKSTRAKSI
"""

#def feature_extraction_ez(signal_array):
#    features = []
#    for signal in signal_array:
#        min_signal, max_signal, mean_signal, std_signal = np.min(signal), np.max(signal), np.mean(signal), np.std(signal)
#        feature = [min_signal, max_signal, mean_signal, std_signal]
#        features.append(feature)
#    return features
#
#def feature_extraction_medium(signal_array,window_count=4,sample_rate=500):
#    features = []
#    for signal in signal_array:
#        min_signal, max_signal, mean_signal, std_signal = np.min(signal), np.max(signal), np.mean(signal), np.std(signal)
#        peaks = find_signal_peaks(signal,minimum=max_signal*0.4)
#        rr_list, rr_startstop = get_rr(peaks,to_sec=True,sample_rate=sample_rate)
#        middle = int(len(rr_list)/2)
#        
#        if (len(rr_list[middle:middle+window_count]) < window_count):
#            rr_list = rr_list[middle-1:middle+window_count-1]
#        else:
#            rr_list = rr_list[middle:middle+window_count]
#        
#        feature = [min_signal, max_signal, mean_signal, std_signal]
#        feature = np.concatenate([feature,rr_list])
#        features.append(feature)
#    return features
#

#QT INTERVAL FITUR
def secondDerivative(signal):
    new = np.zeros_like(signal)
    for i in range(1,len(signal)-1):
        new[i] = (signal[i+1] - (2 * signal[i]) + signal[i-1]) / (len(signal)**2)
    return new   

def detect_qt(data_peak ,to_sec=True, sample_rate=500):
    qt_interval = []
    start_stop = []
    for i in range(1,len(data_peak)-1):
        peak_sebelum = data_peak[i-1]
        peak = data_peak[i]
        peak_sesudah = data_peak[i+1]
        
        
        deteksi_15persen  = (peak-peak_sebelum)*0.15
#        deteksi_15persen  = peak*0.15
        peak_q = round(peak - deteksi_15persen)
        
        deteksi_40persen = (peak_sesudah-peak)*0.40
#        deteksi_40persen = peak*0.40

        peak_t = round(peak + deteksi_40persen)
        qt_interval.append(peak_t-peak_q)
        start_stop.append([peak_q,peak_t])
    if(to_sec):
        qt_interval = np.divide(qt_interval,sample_rate)
    return qt_interval , start_stop



def feature_extraction_qt(signal_array,window_count=6,sample_rate=500, preprocess=True):
    features = []
    for signal in signal_array:
        if (preprocess):
            second = secondDerivative(signal)
            signal = preprocess_ppg_signal(second)
            signal *= 10**10
            
        peaks = find_signal_peaks(signal,minimum=0.2)
        qt_list, qt_startstop = detect_qt(peaks,to_sec=True,sample_rate=sample_rate)

        middle = int(len(qt_list)/2)
        kiri = middle-(window_count//2)
        qt_list = qt_list[kiri:kiri+window_count].tolist()
#        print(len(rr_list))
        if (len(qt_list) == window_count):
            features.append(qt_list)
    return features

import math
#time domain features
def feature_extraction_time_domain_features(signal_array,sample_rate=500 , preprocess=False):
    features = []
    i = 0
    for signal in signal_array:
        if (preprocess):
            signal = preprocess_ppg_signal(signal,sample_rate=sample_rate)
            
        peaks = find_signal_peaks(signal,minimum=0.2)
        rr_list, rr_startstop = get_rr(peaks,to_sec=True,sample_rate=sample_rate)
        
        rata_ppi = np.mean(rr_list)
        std_ppi = np.std(rr_list)
        rata_sinyal = np.mean(signal)
        std_sinyal = np.std(signal)
        
        if(math.isnan(rata_ppi) or math.isnan(std_ppi) or math.isnan(rata_sinyal) or math.isnan(std_sinyal)):
          print("TRUE")
          print(i)
          rata_ppi = np.nansum(np.mean(rr_list))
          std_ppi = np.nansum(np.std(rr_list))
          rata_sinyal = np.nansum(np.mean(signal))
          std_sinyal = np.nansum(np.std(signal))
          features.append([rata_ppi,std_ppi,rata_sinyal,std_sinyal])
        else:
            features.append([rata_ppi,std_ppi,rata_sinyal,std_sinyal])
        i = i+1
    return features

#QT interval dengan sinyal second derivative
    
    
#SLIDING WINDOW
def feature_extraction_pp(signal_array,window_count=6,sample_rate=500, preprocess=False):
    features = []
    for signal in signal_array:
        if (preprocess):
            signal = preprocess_ppg_signal(signal,sample_rate=sample_rate)
            
        peaks = find_signal_peaks(signal,minimum=0.2)
        rr_list, rr_startstop = get_rr(peaks,to_sec=True,sample_rate=sample_rate)
        
        middle = int(len(rr_list)/2)
        kiri = middle-(window_count//2)
        rr_list = rr_list[kiri:kiri+window_count].tolist()
#        print(len(rr_list))
        if (len(rr_list) == window_count):
            features.append(rr_list)
    return features
#END SLIDING WINDOW
    
#SKENARIO 2
#TIME DOMAIN + SLIDING WINDOW
def feature_extraction_time_domain_features_and_sliding_window(signal_array,window_count=6,sample_rate=500 , preprocess=False):
    features = []
    for signal in signal_array:
        if (preprocess):
            signal = preprocess_ppg_signal(signal,sample_rate=sample_rate)
            
        peaks = find_signal_peaks(signal,minimum=0.2)
        rr_list, rr_startstop = get_rr(peaks,to_sec=True,sample_rate=sample_rate)
        
#        TIME DOMAIN
        rata_ppi = np.mean(rr_list)
        std_ppi = np.std(rr_list)
        rata_sinyal = np.mean(signal)
        std_sinyal = np.std(signal)
        
#        SLIDING WINDOW
        middle = int(len(rr_list)/2)
        kiri = middle-(window_count//2)
        sliding_window = rr_list[kiri:kiri+window_count].astype("float64")
        
        if(math.isnan(rata_ppi) or math.isnan(std_ppi) or math.isnan(rata_sinyal) or math.isnan(std_sinyal)):
          print("TRUE")
          
          rata_ppi = np.nansum(np.mean(rr_list))
          std_ppi = np.nansum(np.std(rr_list))
          rata_sinyal = np.nansum(np.mean(signal))
          std_sinyal = np.nansum(np.std(signal))
          
          if (len(sliding_window) == window_count):
              array_1 = [rata_ppi,std_ppi,rata_sinyal,std_sinyal]
              array_1.extend(sliding_window)
              features.append(array_1)

#          features.append([rata_ppi,std_ppi,rata_sinyal,std_sinyal]+sliding_window)
        else:
            if (len(sliding_window) == window_count):
                  array_1 = [rata_ppi,std_ppi,rata_sinyal,std_sinyal]
                  array_1.extend(sliding_window)
                  features.append(array_1)    

#                features.append([rata_ppi,std_ppi,rata_sinyal,std_sinyal]+sliding_window)

    return features
#END TIME DOMAIN + SLIDING WINDOW
    
#TIME DOMAIN + QT
def feature_extraction_time_domain_features_and_qt(signal_array,window_count=6,sample_rate=500 , preprocess=True):
    features = []
    for signal in signal_array:
        if (preprocess):
            second = secondDerivative(signal)
            signal = preprocess_ppg_signal(second,sample_rate=sample_rate)
            signal *= 10**10
            
        peaks = find_signal_peaks(signal,minimum=0.2)
        qt_list, qt_startstop = detect_qt(peaks,to_sec=True,sample_rate=sample_rate)
        
#        TIME DOMAIN
        rata_qt = np.mean(qt_list)
        std_qt = np.std(qt_list)
        rata_sinyal = np.mean(signal)
        std_sinyal = np.std(signal)
        
#        QT
        middle = int(len(qt_list)/2)
        kiri = middle-(window_count//2)
        qt_list = qt_list[kiri:kiri+window_count].astype("float64")
        
        if(math.isnan(rata_qt) or math.isnan(std_qt) or math.isnan(rata_sinyal) or math.isnan(std_sinyal)):
          print("TRUE")
          
          rata_qt = np.nansum(np.mean(qt_list))
          std_qt = np.nansum(np.std(qt_list))
          rata_sinyal = np.nansum(np.mean(signal))
          std_sinyal = np.nansum(np.std(signal))
          if (len(qt_list) == window_count):
              array_1 = [rata_qt,std_qt,rata_sinyal,std_sinyal]
              array_1.extend(qt_list)
              features.append(array_1)
          
#          features.append([rata_qt,std_qt,rata_sinyal,std_sinyal]+qt_list)
          
        else:
            if (len(qt_list) == window_count):
                array_1 = [rata_qt,std_qt,rata_sinyal,std_sinyal]
                array_1.extend(qt_list)
                features.append(array_1)
    return features
#END TIME DOMAIN + QT
#END SKENARIO 2
    

#SKENARIO 3
#TIME DOMAIN + SLIDING WINDOW + QT INTERVAL
def feature_extraction_time_domain_features_and_sliding_window_and_qt(signal_array,window_count=6,sample_rate=500 , preprocess=True):
    features = []
    for signal in signal_array:
        if (preprocess):
            second = secondDerivative(signal)
            signal = preprocess_ppg_signal(second,sample_rate=sample_rate)
            signal *= 10**10
            
        peaks = find_signal_peaks(signal,minimum=0.2)
        rr_list, rr_startstop = get_rr(peaks,to_sec=True,sample_rate=sample_rate)
        qt_list, qt_startstop = detect_qt(peaks,to_sec=True,sample_rate=sample_rate)

        
#        SLIDING WINDOW
        middle = int(len(rr_list)/2)
        kiri = middle-(window_count//2)
        sliding_window = rr_list[kiri:kiri+window_count].astype("float64")
        
#QT            
        middle = int(len(qt_list)/2)
        kiri = middle-(window_count//2)
        qt_list = qt_list[kiri:kiri+window_count].astype("float64")

#        TIME DOMAIN
        rata_ppi = np.mean(sliding_window)
        std_ppi = np.std(sliding_window)
        rata_qt = np.mean(qt_list)
        std_qt = np.std(qt_list)
        rata_sinyal = np.mean(signal)
        std_sinyal = np.std(signal)
        
        if(math.isnan(rata_ppi) or math.isnan(std_ppi) or math.isnan(rata_qt) or math.isnan(std_qt) or math.isnan(rata_sinyal) or math.isnan(std_sinyal)):
          print("TRUE")
          rata_ppi = np.nansum(np.mean(sliding_window))
          std_ppi = np.nansum(np.std(sliding_window))
          rata_qt = np.nansum(np.mean(qt_list))
          std_qt = np.nansum(np.std(qt_list))
          rata_sinyal = np.nansum(np.mean(signal))
          std_sinyal = np.nansum(np.std(signal))
          if (len(sliding_window) == window_count and len(qt_list) == window_count):
              array_1 = [rata_ppi,std_ppi,rata_qt,std_qt,rata_sinyal,std_sinyal]
              array_1.extend(sliding_window)
              array_1.extend(qt_list)
              features.append(array_1)
#          features.append([rata_ppi,std_ppi,rata_qt,std_qt,rata_sinyal,std_sinyal]+sliding_window+qt_list)
          
        else:
            if (len(sliding_window) == window_count and len(qt_list) == window_count):
                array_1 = [rata_ppi,std_ppi,rata_qt,std_qt,rata_sinyal,std_sinyal]
                array_1.extend(sliding_window)
                array_1.extend(qt_list)
                features.append(array_1)
                
#                features.append([rata_ppi,std_ppi,rata_qt,std_qt,rata_sinyal,std_sinyal]+sliding_window+qt_list)
    return features
#END TIME DOMAIN + SLIDING WINDOW + QT INTERVAL


def save_feature_to_csv(file_path,number, ppg_feature):
    df_feature = pd.DataFrame(ppg_feature)
#    df_class = pd.DataFrame(ppg_class)
    df_feature.to_csv("%s/%s.csv" % (file_path, number),index=False,header=False)
#    df_class.to_csv("%s/%sclass.csv" % (file_path, number) ,index=False,header=False)
#def feature_extraction_pp(signal_array,window_count=4,sample_rate=500, preprocess=False):
#    features = []
#    for signal in signal_array:
#        if (preprocess):
#            signal = preprocess_ppg_signal(signal,sample_rate=sample_rate)
#            
#        peaks = find_signal_peaks(signal,minimum=0.2)
#        rr_list, rr_startstop = get_rr(peaks,to_sec=True,sample_rate=sample_rate)
##        middle = int(len(rr_list)/2)
#        
##        if (len(rr_list[middle:middle+window_count]) < window_count):
##            rr_list = rr_list[middle-1:middle+window_count-1]
##        else:
##            rr_list = rr_list[middle:middle+window_count]
##        
##        print(rr_list)
##        feature = np.concatenate([feature,rr_list])
#        features.append(rr_list)
#    return features
#    
"""
END FITUR EKSTRAKSI
"""

def class_count(r_class):
#    count = BeatCount()
    kelas = [0,0,0]
    for cl in r_class:
        if (cl == "N"):
            kelas[0] += 1
        elif (cl == "A"):
            kelas[1] += 1
        elif (cl == "V"):
            kelas[2] += 1
    return kelas


"""
Metrics
"""
def generate_original_and_predicted_class(original_class,predicted_class):
    temp = [0,0,0,0,0,0]
    for i in range(len(original_class)):
        if (original_class[i] == "N"):
            temp[0] += 1
        elif (original_class[i] == "A"):
            temp[1] += 1
        elif (original_class[i] == "V"):
            temp[2] += 1
        if (predicted_class[i] == "N"):
            temp[3] += 1
        elif (predicted_class[i] == "A"):
            temp[4] += 1
        elif (predicted_class[i] == "V"):
            temp[5] += 1
    return temp

def generate_confusion_matrix(number,original_class,predicted_class,smoothing=False,smoothing_value=10^-4):
    count_original = class_count(original_class)
    count_predicted = class_count(predicted_class)
    tp_PAC, tn_PAC, fp_PAC, fn_PAC = 0,0,0,0
    tp_PVC,tn_PVC,fp_PVC,fn_PVC = 0,0,0,0
    acc_PAC, acc_PVC = 0,0
    sp_PAC, sp_PVC = 0,0
    sn_PAC, sn_PVC = 0,0
    f1_PAC, f1_PVC = 0,0
    minimum_number = smoothing_value
    for i in range(len(original_class)):
        ori = original_class[i]
        tes = predicted_class[i]
        if (ori == "N"):
            if (tes == "V"):
                fp_PVC += 1
                tn_PAC += 1
            elif (tes == "A"):
                fp_PAC += 1
                tn_PVC += 1
            else:
                tn_PVC += 1
                tn_PAC += 1
        elif (ori == "V"):
            if (tes == "V"):
                tp_PVC += 1
                tn_PAC += 1
            elif (tes == "A"):
                fp_PAC += 1
                fn_PVC += 1
            else:
                tn_PAC += 1
                fn_PVC += 1
        elif (ori == "A"):
            if (tes == "V"):
                fp_PVC += 1
                fn_PAC += 1
            elif (tes == "A"):
                tp_PAC += 1
                tn_PVC += 1
            else:
                tn_PVC += 1
                fn_PAC += 1
    if (smoothing):
        acc_PAC = (tn_PAC+tp_PAC+minimum_number) / (tn_PAC+tp_PAC+fn_PAC+fp_PAC+minimum_number) * 100
        acc_PVC = (tn_PVC+tp_PVC+minimum_number) / (tn_PVC+tp_PVC+fn_PVC+fp_PVC+minimum_number) * 100
        sp_PAC = (tn_PAC+minimum_number)/(tn_PAC+fp_PAC+minimum_number) * 100
        sp_PVC = (tn_PVC+minimum_number)/(tn_PVC+fp_PVC+minimum_number) * 100
        sn_PAC = (tp_PAC+minimum_number) / (tp_PAC+fn_PAC+minimum_number) * 100
        sn_PVC = (tp_PVC+minimum_number) / (tp_PVC+fn_PVC+minimum_number) * 100
    else:
        try:
            acc_PAC = (tn_PAC+tp_PAC) / (tn_PAC+tp_PAC+fn_PAC+fp_PAC) * 100
        except:
            acc_PAC = 0
        try:
            acc_PVC = (tn_PVC+tp_PVC) / (tn_PVC+tp_PVC+fn_PVC+fp_PVC) * 100
        except:
            acc_PVC = 0
        try: 
            sp_PAC = tn_PAC/(tn_PAC+fp_PAC) * 100
        except:
            sp_PAC = 0
        try:
            sp_PVC = tn_PVC/(tn_PVC+fp_PVC) * 100
        except:
            sp_PVC = 0
        try:
            sn_PAC = (tp_PAC) / (tp_PAC+fn_PAC) * 100
        except:
            sn_PAC = 0
        try:
            sn_PVC = (tp_PVC) / (tp_PVC+fn_PVC) * 100
        except:
            sn_PVC = 0
    temp = [number,count_original[0], count_original[1], count_original[2], count_predicted[0], count_predicted[1], count_predicted[2], tp_PAC, fp_PAC, tn_PAC, fn_PAC,round(acc_PAC,2), round(sp_PAC,2),sn_PAC, tp_PVC, fp_PVC, tn_PVC, fn_PVC,round(acc_PVC,2),round(sp_PVC,2),sn_PVC]
    columns = ["Number", "Normal","PAC","PVC","pred_Normal", "pred_PAC","pred_PVC","tp_PAC","fp_PAC","tn_PAC","fn_PAC","acc_PAC","sp_PAC","sn_PAC","tp_PVC", "fp_PVC", "tn_PVC", "fn_PVC","acc_PVC","sp_PVC","sn_PVC"]
#    print(len(temp), len(columns))
    return temp

# DWT EXTRACTION

In [ ]:
import numpy as np
import csv, os, glob, ntpath
from dwt import dwt
import pandas as pd
from libraries import *

x = np.array([])
y = np.array([])
path = 'D:\\TA ORANG\\Ryan\\test\\data1'
pathname = "D:\\TA ORANG\\Ryan\\test\\data1\\"
#os.chdir(path)
#files = glob.glob('*.{}'.format('csv'))
files = glob.glob(os.path.join(path, '*.csv'))
file_path = "fitur/dwt/"

##Reading file and saving to variables
for filename in files:
    with open(filename) as csvfile:
        print(filename)
        f_read = csv.reader(csvfile, delimiter = ',')
        number = filename.replace(".csv","")
        number = number.replace(pathname,"")
        hasilfile = file_path+number+".csv"
        csv_out = open(hasilfile, "w")  
        print(hasilfile)
        #ppg_signal, ppg_class = annotation_to_ppg_signal_labeled("data","annotation",number)
        #print(ppg_signal)
        for row in f_read:
            if (row[0] != "'sample interval'" and row[0] != "'0.002 sec'") : 
                #taking x parameter only for plotting the signal
                #x = np.append(x, [float(j) for j in row]) -- Comented as only using 1 value
                x = np.append(x, float(row[0]))

                #Cleaning signal data
                if row[2][:1] == '-' : 
                    if row[2][1:len(row[2])] == '' :
                        y = np.append(y, 0)
                    else :
                        curr = float(row[2][1:len(row[2])])
                        curr = float(-curr)
                        y = np.append(y, curr)
                else :
                    y = np.append(y, float(row[2]))

        feature_dwt = dwt(y)
        feature_dwt1=np.array(feature_dwt.flatten())
        a_str=','.join(str(x) for x in feature_dwt1)
        print(a_str)
        csv_out.write(a_str)
        #save_feature_to_csv(file_path,number,a_str)
        #dwt_feature = pd.DataFrame(feature_dwt)
        #a_str.to_csv("%s/%s.csv" % (file_path, number),index=False,header=False)
        #print(feature_dwt)
        csv_out.close()

# **PPG FeatureExtraction**





In [ ]:
#from scipy.signal import argrelmax, argrelmin
import numpy as np 
from libraries import *
import os
from time import time
"""
Main Program
"""

#ppg_normal, ppg_pac, ppg_pvc = annotation_to_ppg_signal("/home/ipat/Project/TA/KODING_FINAL/data","/home/ipat/Project/TA/KODING_FINAL/annotation","212")
#all_feature = []

lokasi_file = "fitur/gabungan"

for filename in os.listdir("annotation"):
    if (".csv" in filename):
        start = time()
        nomor = filename.replace(".csv","")
        print(nomor)
        try:
            ppg_signal, ppg_class = annotation_to_ppg_signal_labeled("data","annotation",nomor)
            print(ppg_signal)
#            ganti ganti
            ppg_feature = feature_extraction_time_domain_features_and_sliding_window_and_qt(ppg_signal,preprocess=True)

            for i in range(len(ppg_feature)):
                ppg_feature[i].append(ppg_class[i])
            save_feature_to_csv(lokasi_file,nomor,ppg_feature)
            print(time() - start, "second "+lokasi_file)
        except Exception as e:
            print("failed",e)
            print(time() - start, "second "+lokasi_file)
            

# *PPG LEARNING FEATURE*

In [ ]:
#from scipy.signal import argrelmax, argrelmin
import numpy as np 
from libraries import *
import os
from time import time
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
"""
Generate Features and Classes
"""
all_feature, all_class = [], []

skenario = "fitur/"
nama_fitur = "dwt"

fitur_path = skenario+nama_fitur
model_path = "model_skenario/"+skenario
model_filename = "KNN_7_"+nama_fitur

for filename in os.listdir(fitur_path): 
    if (".csv" in filename):
        start = time()
        nomor = filename.replace(".csv","")
        print(nomor)
        try:
            feature_list, class_list = read_feature_from_csv(fitur_path+"/%s.csv" % nomor)
            if (len(all_feature) == 0):
                all_feature = feature_list
                all_class= class_list
            else:
                all_feature = np.concatenate([all_feature, feature_list])
                all_class = np.concatenate([all_class, class_list])
#            print (class_count(class_list))
            print(time() - start, "second")
        except Exception as e:
            print("failed",e)
            print(time() - start, "second")


"""
Split Train and Test
"""            
from imblearn.over_sampling import SMOTE
ros = SMOTE()
X_resampled, y_resampled = ros.fit_resample(all_feature,all_class)
count_original = class_count(all_class)
count_resampled = class_count(y_resampled)

"""
kFold Algorithm
"""
from sklearn.model_selection import KFold # import KFold
kf = KFold(n_splits=10) # Define the split - into 2 folds 
accuracy_list = []
for train_index, test_index in kf.split(X_resampled):
    X_train, X_test = X_resampled[train_index], X_resampled[test_index]
    y_train, y_test = y_resampled[train_index], y_resampled[test_index]
    
    """
    Import and fit classifier
    """
    #import skflow
    #
#    aktivasi = "tanh"
#    sizes = 12
#    clf = MLPClassifier(activation=aktivasi,verbose=1,hidden_layer_sizes=12,max_iter=300)
    #clf = LogisticRegression()
    clf = KNeighborsClassifier(n_neighbors=7)
    
    clf.fit(X_train,y_train)
    pred = clf.predict(X_test)
    """
    Training Metrics
    """
    accuracy = accuracy_score(y_test,pred)
    accuracy_list.append(accuracy)


average_accuracy = np.average(accuracy_list)
accuracy_list.append(average_accuracy)

akurasi_tocsv=pd.DataFrame(accuracy_list)
akurasi_tocsv.to_csv("%s_akurasi.csv" % (model_path+model_filename),index=False,header=False)
print(average_accuracy)
"""
Save Model To File using Pickle
"""
import pickle
#pickle.dump(clf,open(model_path+"/logistic_regression.sav","wb"))
#pickle.dump(clf,open(model_path+"/ANN_%s_%s.sav" % (sizes, aktivasi),"wb"))
#pickle.dump(clf,open(model_path+"/decision_tree.sav","wb"))
#pickle.dump(clf,open(model_path+"/decision_tree.sav","wb"))

pickle.dump(clf,open(model_path+model_filename+".sav","wb"))
#hehe = generate_original_and_predicted_class(y_test,pred)


# PPG PredictFromModel

In [ ]:
#from scipy.signal import argrelmax, argrelmin
import numpy as np 
from libraries import *
import os
from time import time

"""
Import classifier
"""
skenario = "fitur_skenario_3/"
nama_fitur = "gabungan"

fitur_path = skenario+nama_fitur
lokasi_model = "model_skenario/"+skenario

import pickle
clf = pickle.load(open(lokasi_model+"/KNN_7_"+nama_fitur+".sav","rb"))
"""
Generate Features and Classes
"""
hasil = []
for filename in os.listdir(fitur_path):
    if (".csv" in filename):
        start = time()
        nomor = filename.replace(".csv","")
        print(nomor)
        try:
            feature_list, class_list = read_feature_from_csv(fitur_path+"/%s.csv" % nomor)
            pred = clf.predict(feature_list)
            hasil.append(generate_confusion_matrix(nomor,class_list,pred,smoothing=False))
        except Exception as e:
            print("failed",e)
            print(time() - start, "second")

"""
Export Result to CSV
"""
columns_list = ["Sample_Data", "Normal","PAC","PVC","pred_Normal", "pred_PAC","pred_PVC","True_Positive_PAC","False_Positive_PAC","True_Negative_PAC","False_Negative_PAC","Accuracy_PAC","Spesifisity_PAC","sn_PAC","True_Positive_PVC", "False_Positive_PVC", "True_Negative_PVC", "False_Negative_PVC","Accuracy_PVC","Spesifisity_PVC","sn_PVC"]
pd.DataFrame(hasil,columns=columns_list).to_csv("hasil_skenario/"+fitur_path+"/confusion_matrix_fitur_"+nama_fitur+".csv",index=False)

data = pd.DataFrame(hasil)